In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
nvi_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/NVI Zones/nvi_neighborhood_zones_temp_2025.shp")
detroit_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/City_of_Detroit_Boundary/City_of_Detroit_Boundary.shp")
cd_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/Detroit_City_Council_Districts_2026/Detroit_City_Council_Districts_2026.shp")

In [3]:
# Load birth data from csv
births_df = pd.read_csv('Q:/Vital Records/2023/2023 Birth Files.csv', low_memory=False)

In [4]:
# Assuming the birth data contains 'latitude' and 'longitude' columns
# Convert to a GeoDataFrame
births_gdf = gpd.GeoDataFrame(
    births_df, 
    geometry=gpd.points_from_xy(births_df.LONGITUDE, births_df.LATITUDE), 
    crs="EPSG:4326")

### Detroit - City Wide

In [5]:
# Ensure both GeoDataFrames have the same CRS
births_gdf = births_gdf.to_crs(detroit_shp.crs)

# Perform a spatial join to assign births to geographic areas
births_with_geography = gpd.sjoin(births_gdf, detroit_shp, how="left", predicate="within")

#No of rows in the data
total_births = births_with_geography.groupby("name")["KESSNER"].count().reset_index()
total_births.columns = ["name", "total_births"]

# Filter for births where kesser == 1
births_kesser_1 = births_with_geography[births_with_geography["KESSNER"] == 1]

# Aggregate count of kesser = 1 by geographic area (e.g., 'zone' column from shapefile)
adequate_care_counts = births_kesser_1.groupby("name")["KESSNER"].count().reset_index()
adequate_care_counts.columns = ["name", "kessner_1_count"]

births_summary = total_births.merge(adequate_care_counts, on="name", how="left")
births_summary["percentage_adequate"] = (births_summary["kessner_1_count"] / births_summary["total_births"]) * 100

display(births_summary)

,name,total_births,kessner_1_count,percentage_adequate
0,Detroit,7426,4594,61.863722


### NVI Zones

In [12]:
# Ensure both GeoDataFrames have the same CRS
births_gdf = births_gdf.to_crs(nvi_shp.crs)

# Spatial join: Assign each point to a polygon
merged_gdf = gpd.sjoin(births_gdf, nvi_shp, how="left", predicate="intersects")

#No of rows in the data
total_births_nvi = merged_gdf.groupby(["district_n", "zone_id"])["KESSNER"].count().reset_index()
total_births_nvi.columns = ["district_n", "zone_id", "total_births"]

# Filter for births where kesser == 1
births_kesser_1_nvi = merged_gdf[merged_gdf["KESSNER"] == 1]

# Aggregate count of kesser = 1 by geographic area (e.g., 'zone' column from shapefile)
adequate_care_counts_nvi = births_kesser_1_nvi.groupby(["district_n", "zone_id"])["KESSNER"].count().reset_index()
adequate_care_counts_nvi.columns = ["district_n", "zone_id", "kessner_1_count"]

births_summary_nvi = total_births_nvi.merge(adequate_care_counts_nvi, on=["district_n","zone_id"], how="left")
births_summary_nvi["percentage_adequate"] = (births_summary_nvi["kessner_1_count"] / births_summary_nvi["total_births"]) * 100

display(births_summary_nvi)

,district_n,zone_id,total_births,kessner_1_count,percentage_adequate
0,1,1a,383,254,66.318538
1,1,1b,236,142,60.169492
2,1,1c,340,230,67.647059
3,2,2a,426,284,66.666667
4,2,2b,211,144,68.246445
5,2,2c,293,179,61.092150
6,3,3a,475,302,63.578947
7,3,3b,445,239,53.707865
8,3,3c,341,231,67.741935
9,4,4a,518,281,54.247104


### Council Districts 2026

In [10]:
# Ensure both GeoDataFrames have the same CRS
births_gdf_cd = births_gdf.to_crs(cd_shp.crs)

# Perform a spatial join to assign births to geographic areas
births_with_cd = gpd.sjoin(births_gdf_cd, cd_shp, how="left", predicate="within")

#No of rows in the data
total_births_cd = births_with_cd.groupby("council_di")["KESSNER"].count().reset_index()
total_births_cd.columns = ["council_di", "total_births"]

# Filter for births where kesser == 1
births_kesser_1_cd = births_with_cd[births_with_cd["KESSNER"] == 1]

# Aggregate count of kesser = 1 by geographic area (e.g., 'zone' column from shapefile)
adequate_care_counts_cd = births_kesser_1_cd.groupby("council_di")["KESSNER"].count().reset_index()
adequate_care_counts_cd.columns = ["council_di", "kessner_1_count"]

births_summary_cd = total_births_cd.merge(adequate_care_counts_cd, on="council_di", how="left")
births_summary_cd["percentage_adequate"] = (births_summary_cd["kessner_1_count"] / births_summary_cd["total_births"]) * 100

display(births_summary_cd)

,council_di,total_births,kessner_1_count,percentage_adequate
0,1.0,958,625,65.240084
1,2.0,926,605,65.334773
2,3.0,1266,775,61.216430
3,4.0,1042,570,54.702495
4,5.0,821,523,63.702801
5,6.0,1097,650,59.252507
6,7.0,1316,846,64.285714
